# SSLTelecom

## Problema

Te escribimos de la peluquería SSL, esta pandemia – entre otras cosas –, ha afectado a nuestro negocio, que a veces parece que está a punto de cerrar, para nuestro disgusto. Como no queremos rendirnos tan fácilmente, hemos pensado en abrir un nuevo negocio de **venta de teléfonos móviles** para ver si sacamos suficientes beneficios para mantener la pelu a flote.

Supongo que a estas alturas, te estarás preguntando: ¿y yo qué pinto en el negocio? Pues la verdad, es que no sabemos muy bien qué precio ponerle a nuestros teléfonos móviles y no queremos quedar atrás de nuestros competidores. Es por eso que necesitamos la ayuda de un **orfebre Data Scientist** que nos ayude a estimar correctamente el rango de precio de un teléfono móvil (bajo/medio/alto) para nosotras podernos hacer una idea del precio final al que podemos venderlo y no perder dinero.

Hemos adjuntado el fichero con los datos que tenemos disponibles actualmente. Si necesitas un diccionario con información relativa a los datos, háznoslo saber.

## Datos

- **Battery power**: indica la batería del móvil expresada en mAh
- **Blue**: indica si el móvil tiene bluetooth (1) o no (0)
- **Clock speed**: indica la rapidez de procesamiento
- **Dual sim**: indica si el móvil soporta dos SIM (1) o no (0)
- **Fc**: megapíxeles de la cámara frontal
- **Four_g**: indica si el móvil tiene 4G (1) o no (0)
- **Int_memory**: memoria interna expresada en Gb
- **M_dep**: grosor del teléfono móvil
- **Mobile_wt**: peso del teléfono móvil
- **N_cores**: número de cores del procesador
- **Pc**: megapíxeles de la cámara principal
- **Px_height**: resolución de píxeles (altura)
- **Px_width**: resolución de píxeles (ancho)
- **Ram**: RAM del móvil expresada en MegaBytes
- **Sc_h**: altura de la pantalla del móvil expresada en cm
- **Sc_w**: ancho de la pantalla del móvil expresada en cm
- **Talk time**: duración de la batería durante una conversación
- **Three_g**: indica si el móvil tiene 3G (1) o no (0)
- **Touch_screen**: indica si la pantalla del móvil es táctil (1) o no (0)
- **WiFi**: indica si el móvil tiene WiFi (1) o no (0)
- **Price-range**: indica si el móvil es de costo bajo (0), medio (1), alto (2) o muy alto (3)

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd

DATA_FILEPATH = '../data/mobile_phone_info.csv'
df = pd.read_csv(DATA_FILEPATH)
df.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,...,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,...,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,...,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,...,1208,1212,1411,8,2,15,1,1,0,1


In [4]:
from sklearn.model_selection import train_test_split

y = df['price_range']
X = df.drop(['price_range'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [6]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)

LinearRegression()

In [8]:
import numpy as np
from sklearn.metrics import mean_squared_error

predictions = lin_reg.predict(X_train)
lin_mse = mean_squared_error(y_train, predictions)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

0.31762903653181246

In [11]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

quantitative_columns = ['battery_power','clock_speed', 'fc', 'int_memory', 'm_dep', 'mobile_wt',
 'n_cores', 'pc', 'px_height', 'px_width', 'ram', 'sc_h', 'sc_w','talk_time']
qualitative_columns = ['blue', 'dual_sim', 'four_g', 'three_g', 'touch_screen', 'wifi']

df_num = df[quantitative_columns]

num_pipeline = Pipeline([
        ('std_scaler', StandardScaler()),
    ])

df_num_tr = num_pipeline.fit_transform(df_num)

In [12]:
from sklearn.compose import ColumnTransformer

num_attribs = list(df_num)

full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attribs),
    ])

df_prepared = full_pipeline.fit_transform(df)